In [1]:
import os
import dspy
import mlflow
import pandas as pd
from dotenv import load_dotenv
import kagglehub
import ujson

/Users/viktorsvan/Python_koder/ingenjor_A_och_maskininlarnings_kod/LLM-DSPy-project/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
lm = dspy.LM("gpt-4o", api_key=openai_api_key)

In [3]:
local_lm = dspy.LM("ollama_chat/llama3.2", api_base="http://localhost:11434", api_key="")
dspy.configure(lm=local_lm)

In [4]:
class Chat(dspy.Signature):
    "You are a helpful assistant that answer questions about cats and dogs."
    question: str = dspy.InputField(desc="Questions asked by the user")
    response: str = dspy.OutputField(desc="Response to the question")

class Model(dspy.Module):
    def __init__(self):
        super().__init__()
        self.respond = dspy.ChainOfThought(Chat)

    def forward(self, question: str):
        return self.respond(question=question)

In [5]:
model = Model()

In [6]:
#mlflow ui --port 5000
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("DSPy")
mlflow.dspy.autolog()

Laddar ner dataset med beskrivning: "Contains 583 unique questions and answers related to dogs and cats.
It includes questions related to health, training, lifestyle, breeds and commonly asked questions."

In [7]:
path = kagglehub.dataset_download("bishnushahi/dog-cat-qa")

In [8]:

csv_path = os.path.join(path, 'Dog-Cat-QA.csv')

In [9]:
df = pd.read_csv(csv_path)
print(df.head())

   Unnamed: 0                                           Question  \
0           0  What are some examples of breed-typical person...   
1           1  How do working and herding dogs typically behave?   
2           2   Which breed of dogs is known for strong loyalty?   
3           3  What instincts do guarding dogs typically disp...   
4           4  How does breed specificity impact a dog's abil...   

                                              Answer  
0  Breed-typical personalities in dogs are develo...  
1  Working and herding dogs have business-like di...  
2  Collies and Akitas are known for their strong ...  
3  Guarding dogs tend to be protective of their t...  
4  Breed specificity affects how well dogs adapt ...  


In [10]:
df = df.drop('Unnamed: 0', axis=1)
print(df.head())

                                            Question  \
0  What are some examples of breed-typical person...   
1  How do working and herding dogs typically behave?   
2   Which breed of dogs is known for strong loyalty?   
3  What instincts do guarding dogs typically disp...   
4  How does breed specificity impact a dog's abil...   

                                              Answer  
0  Breed-typical personalities in dogs are develo...  
1  Working and herding dogs have business-like di...  
2  Collies and Akitas are known for their strong ...  
3  Guarding dogs tend to be protective of their t...  
4  Breed specificity affects how well dogs adapt ...  


In [11]:
df.columns = ['question', 'response']

Gör om det till dictionary par:

In [12]:
data = df.to_dict(orient='records')
data = [dspy.Example(**d).with_inputs('question') for d in data]

In [13]:
example = data[2]
example

Example({'question': 'Which breed of dogs is known for strong loyalty?', 'response': 'Collies and Akitas are known for their strong sense of loyalty.'}) (input_keys={'question'})

Till skillnad från vanlig ML träning: Så ska man dela upp datasetet med 20 procent till träning och 80 procent till validering. 

"For prompt optimizers in particular, it's often better to pass more validation than training.
20% training and 80% validation.

In [14]:
print(len(data))

583


In [15]:
import random

random.Random(0).shuffle(data) 

trainset = data[:116]   # 20% 
valset   = data[116:466]  # 60% 
testset  = data[466:583]     # 20% 

len(trainset), len(valset), len(testset)

(116, 350, 117)

Nu laddar jag in en Evaluation metric som i sig också är en DSPy module: 

In [16]:
from dspy.evaluate import SemanticF1

In [17]:
metric = SemanticF1(decompositional=True)

# Produce a prediction from our `cot` module, using the `example` above as input.
pred = model(**example.inputs())

# Compute the metric score for the prediction.
score = metric(example, pred)

print(f"Question: \t {example.question}\n")
print(f"Gold Response: \t {example.response}\n")
print(f"Predicted Response: \t {pred.response}\n")
print(f"Semantic F1 Score: {score:.2f}")

Question: 	 Which breed of dogs is known for strong loyalty?

Gold Response: 	 Collies and Akitas are known for their strong sense of loyalty.

Predicted Response: 	 Some specific breeds that are known for strong loyalty include German Shepherds, Doberman Pinschers, and Rottweilers. These breeds have been bred for centuries to be protective of their families and territories, making them highly devoted companions.

Semantic F1 Score: 0.86


[Trace(request_id=9216c5f4fe3a4f51933394b284533b08), Trace(request_id=01da9c3a94e945e7885a91871615a042)]

In [18]:
dspy.inspect_history(n=1)





[2025-05-18T07:26:37.077033]

System message:

Your input fields are:
1. `question` (str)
2. `ground_truth` (str)
3. `system_response` (str)
Your output fields are:
1. `reasoning` (str)
2. `ground_truth_key_ideas` (str): enumeration of key ideas in the ground truth
3. `system_response_key_ideas` (str): enumeration of key ideas in the system response
4. `discussion` (str): discussion of the overlap between ground truth and system response
5. `recall` (float): fraction (out of 1.0) of ground truth covered by the system response
6. `precision` (float): fraction (out of 1.0) of system response covered by the ground truth
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## ground_truth ## ]]
{ground_truth}

[[ ## system_response ## ]]
{system_response}

[[ ## reasoning ## ]]
{reasoning}

[[ ## ground_truth_key_ideas ## ]]
{ground_truth_key_ideas}

[[ ## system_response_key_ideas ## ]]
{system_response_k

In [19]:
evaluate = dspy.Evaluate(devset=trainset, metric=metric,
                         display_progress=True, display_table=2, )

In [20]:
evaluate(model)

Average Metric: 77.25 / 116 (66.6%): 100%|██████████| 116/116 [00:01<00:00, 64.59it/s]

2025/05/18 07:26:38 INFO dspy.evaluate.evaluate: Average Metric: 77.25493796462922 / 116 (66.6%)


,question,example_response,reasoning,pred_response,SemanticF1
0,Describe the distinctive appearance and temperament of the Birman ...,: The Birman cat is colorpointed with long silky hair and four pur...,The Birman cat is a breed known for its distinctive appearance and...,The Birman cat is a breed that combines striking appearance with a...,✔️ [0.774]
1,What other considerations should pet owners keep in mind when choo...,"Pet owners should consider factors like cost, refrigeration requir...","When choosing between wet and dry cat food, pet owners should also...","When choosing between wet and dry cat food, consider your cat's in...",✔️ [0.667]


66.6

[Trace(request_id=1bce7f3d17ba4835ace8ed55af9d26f0), Trace(request_id=b74150d9975643d695a7f78e84d0604f), Trace(request_id=17f63fd89b334b14bc399597a5a4d27f), Trace(request_id=6f152a09aa244d5bb70c3e90bf4d6e32), Trace(request_id=cd23c9f56485430ab6a9730d744101bf), Trace(request_id=dd40959edef340c2af97b7cf4ea6b357), Trace(request_id=dbe7beab2f6a45e5a78d127907462a51), Trace(request_id=3602adbc26594324be606363a05ca88d), Trace(request_id=23116cd9a6db4eb7a65b92a585dc8363), Trace(request_id=770b364e13834f73a765ecdf492ced3a)]

In [21]:
from dspy.teleprompt import MIPROv2

optimizer = MIPROv2(
    metric=SemanticF1, 
    num_threads=1
)

print("Optimizing zero-shot program MIPROv2...")
zero_shot_program = optimizer.compile(
    model.deepcopy(),
    trainset=trainset,
    valset=valset,
    requires_permission_to_run=False,
)

2025/05/18 07:26:39 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 10
minibatch: True
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 100

2025/05/18 07:26:39 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/05/18 07:26:39 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/05/18 07:26:39 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Optimizing zero-shot program MIPROv2...
Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


  0%|          | 0/116 [00:00<?, ?it/s]2025/05/18 07:26:39 ERROR dspy.teleprompt.bootstrap: Failed to run or to evaluate example Example({'question': 'Describe the distinctive appearance and temperament of the Birman cat.', 'response': ': The Birman cat is colorpointed with long silky hair and four pure white feet. It has a strong head with a firm chin and is friendly, affectionate, and docile.'}) (input_keys={'question'}) with <class 'dspy.evaluate.auto_evaluation.SemanticF1'> due to SemanticF1.__init__() takes from 1 to 3 positional arguments but 4 were given.
2025/05/18 07:26:39 ERROR dspy.teleprompt.bootstrap: Failed to run or to evaluate example Example({'question': 'What other considerations should pet owners keep in mind when choosing between wet and dry cat food?', 'response': 'Pet owners should consider factors like cost, refrigeration requirements, and shelf life when choosing between wet and dry cat food.'}) (input_keys={'question'}) with <class 'dspy.evaluate.auto_evaluatio

  0%|          | 0/100 [00:00<?, ?it/s]

2025/05/18 07:26:39 ERROR dspy.teleprompt.utils: An exception occurred during evaluation
Traceback (most recent call last):
  File "/Users/viktorsvan/Python_koder/ingenjor_A_och_maskininlarnings_kod/LLM-DSPy-project/venv/lib/python3.11/site-packages/dspy/teleprompt/utils.py", line 51, in eval_candidate_program
    return evaluate(candidate_program, devset=trainset, return_all_scores=return_all_scores, callback_metadata={"metric_key": "eval_full"})
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/viktorsvan/Python_koder/ingenjor_A_och_maskininlarnings_kod/LLM-DSPy-project/venv/lib/python3.11/site-packages/mlflow/utils/autologging_utils/safety.py", line 402, in safe_patch_function
    return original(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/viktorsvan/Python_koder/ingenjor_A_och_maskininlarnings_kod/LLM-DSPy-project/venv/lib/python3.11/site-packages/dspy/u

...


  0%|          | 0/35 [00:00<?, ?it/s]

2025/05/18 07:26:39 ERROR dspy.teleprompt.utils: An exception occurred during evaluation
Traceback (most recent call last):
  File "/Users/viktorsvan/Python_koder/ingenjor_A_och_maskininlarnings_kod/LLM-DSPy-project/venv/lib/python3.11/site-packages/dspy/teleprompt/utils.py", line 54, in eval_candidate_program
    return evaluate(
           ^^^^^^^^^
  File "/Users/viktorsvan/Python_koder/ingenjor_A_och_maskininlarnings_kod/LLM-DSPy-project/venv/lib/python3.11/site-packages/mlflow/utils/autologging_utils/safety.py", line 402, in safe_patch_function
    return original(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/viktorsvan/Python_koder/ingenjor_A_och_maskininlarnings_kod/LLM-DSPy-project/venv/lib/python3.11/site-packages/dspy/utils/callback.py", line 343, in sync_wrapper
    raise exception
  File "/Users/viktorsvan/Python_koder/ingenjor_A_och_maskininlarnings_kod/LLM-DSPy-project/venv/lib/python3.11/site-packages/dspy/utils/callback.py", line 339, in sync_wrap

  0%|          | 0/100 [00:00<?, ?it/s]

2025/05/18 07:26:39 ERROR dspy.teleprompt.utils: An exception occurred during evaluation
Traceback (most recent call last):
  File "/Users/viktorsvan/Python_koder/ingenjor_A_och_maskininlarnings_kod/LLM-DSPy-project/venv/lib/python3.11/site-packages/dspy/teleprompt/utils.py", line 51, in eval_candidate_program
    return evaluate(candidate_program, devset=trainset, return_all_scores=return_all_scores, callback_metadata={"metric_key": "eval_full"})
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/viktorsvan/Python_koder/ingenjor_A_och_maskininlarnings_kod/LLM-DSPy-project/venv/lib/python3.11/site-packages/mlflow/utils/autologging_utils/safety.py", line 402, in safe_patch_function
    return original(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/viktorsvan/Python_koder/ingenjor_A_och_maskininlarnings_kod/LLM-DSPy-project/venv/lib/python3.11/site-packages/dspy/u

  0%|          | 0/35 [00:00<?, ?it/s]
